# Imports

In [1]:
from enum import Enum
import numpy as np
import pdb
from typing import Tuple, List
from itertools import product

# Cats

In [ ]:
snowball = [
    [1, 0, 0],
    [1, 1, 1]]

jade_goody = [
    [0, 1, 0],
    [0, 1, 1],
    [1, 1, 1]
]

foxy_lady = [
    [1, 1],
    [1, 1],
    [1, 1]
]

black_panther = [
    [1, 0, 1],
    [1, 1, 1],
    [1, 1, 1]
]

turquoise_shelly = [
    [1, 0],
    [1, 1]
]

lucy_in_the_sky_with_diamonds = [
    [1],
    [1]
]

In [2]:
grid_1 = [
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1]
]

# Transforms

In [3]:
grid = [
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1]
]

class Orientation(Enum):
    IDENTITY = 1
    R_90 = 2
    R_180 = 3
    R_270 = 4
    REF_IDENTITY = 5
    REF_R_90 = 6
    REF_R_180 = 7
    REF_R_270 = 8

def transform_coordinates(coordinates, orientation):
    if orientation == Orientation.IDENTITY:
        return np.array(coordinates)
    elif orientation == Orientation.R_90:
        return np.rot90(coordinates)
    elif orientation == Orientation.R_180:
        return np.rot90(coordinates, 2)
    elif orientation == Orientation.R_270:
        return np.rot90(coordinates, 3)
    elif orientation == Orientation.REF_IDENTITY:
        return np.fliplr(coordinates)
    elif orientation == Orientation.REF_R_90:
        return np.rot90(np.fliplr(coordinates))
    elif orientation == Orientation.REF_R_180:
        return np.rot90(np.fliplr(coordinates), 2)
    elif orientation == Orientation.REF_R_270:
        return np.rot90(np.fliplr(coordinates), 3)

In [4]:
# transform_coordinates(snowball_coord, Orientation.IDENTITY)
transform_coordinates(snowball, Orientation.R_90)

array([[0, 1],
       [0, 1],
       [1, 1]])

In [5]:
transform_coordinates(snowball, Orientation.R_270)

array([[1, 1],
       [1, 0],
       [1, 0]])

In [6]:
transform_coordinates(snowball, Orientation.REF_IDENTITY)

array([[0, 0, 1],
       [1, 1, 1]])

In [7]:
transform_coordinates(snowball, Orientation.REF_R_90)

array([[1, 1],
       [0, 1],
       [0, 1]])

In [8]:
transform_coordinates(snowball, Orientation.REF_R_270)

array([[1, 0],
       [1, 0],
       [1, 1]])

# Sliding Window

In [9]:
def sliding_window(_grid, cat_coord):
    grid = np.array(_grid)
    grid_width = grid.shape[1]
    grid_height = grid.shape[0]
    cat_width = cat_coord.shape[1]
    cat_height = cat_coord.shape[0]
    return [(i, j) for j in range(grid_width - cat_width + 1) for i in range(grid_height - cat_height + 1)]
#     for i in range(grid_height - cat_height + 1):
#         for j in range(grid_width - cat_width + 1):
#             yield (i, j)

In [10]:
sliding_window(grid, transform_coordinates(snowball, Orientation.IDENTITY))

[(0, 0), (1, 0)]

# Visualisation

In [11]:
def cats_fit(_grid, cats: List[Tuple[str, List[List[int]], Tuple[int, int]]]) -> None:
    rendered_grid = np.array(_grid, str)
    for name, coord, xy in cats:
        abbrev = name[0]
        for i, row in enumerate(coord):
            for j, column in enumerate(row):
                if column == 1:
                    rendered_grid[i + xy[0]][j + xy[1]] = abbrev
        print("({}){}".format(name[0], name[1:]))
    print('\n')
    print(rendered_grid)


cats = [
    ('turquoise shelly', turquoise_shelly, (0, 1))
]

cats_fit(grid_1, cats)

(t)urquoise shelly


[['1' 't' '1' '1' '1']
 ['1' 't' 't' '1' '1']
 ['1' '1' '1' '1' '1']
 ['1' '1' '1' '1' '1']
 ['1' '1' '1' '1' '1']]


# Verifi-cat-ion

In [12]:

def verify_grid_cats(_grid, cats: List[Tuple[List[List[int]], Tuple[int, int]]]) -> bool:
    grid = np.array(_grid)
    for coord, xy in cats:
        for i, row in enumerate(coord):
            for j, column in enumerate(row):
                if column == 1 and grid[i + xy[0]][j + xy[1]] == 0:
                    return False
                elif column == 1 and grid[i + xy[0]][j + xy[1]] > 1:
                    return False
                # TODO: need to check for non-occupiable grid spaces
                elif column == 1 and grid[i + xy[0]][j + xy[1]] == 1:
                    grid[i + xy[0]][j + xy[1]] += 1
    return not (1 in grid)

cats = [
    (foxy_lady, (0, 0))
]

In [13]:
verify_grid_cats(foxy_lady, cats), verify_grid_cats(foxy_lady, [(lucy_in_the_sky_with_diamonds, (0, 0))])

(True, False)

# Brute Force Solve

In [14]:
def solve(grid, cats: List[Tuple[str, List[List[int]]]]):
    
    cat_transforms = [[transform_coordinates(coord, t) for t in Orientation] for _, coord in cats]
    names = [name for name, _ in cats]
    cats_t_xys = [[[(coord, xy) 
                    for xy in sliding_window(grid, coord)]
                   for coord in coords]
                  for coords in cat_transforms]
    
    # Combinations of cats in their different orientations
    cats_transforms_product = product(*cats_t_xys)
    for ctp in cats_transforms_product:
        
        # Combinations of cat orientations and their positions
        for coord_xys in product(*ctp):
            if verify_grid_cats(grid, coord_xys):
                name_coord_xys = [(name, coord_xy[0], coord_xy[1]) for name, coord_xy in zip(names, coord_xys)]
                cats_fit(grid, name_coord_xys)
                return
    
    print('no combinations found!')

cats = [
    ('foxy lady', foxy_lady),
    ('black panther', black_panther),
    ('jade goody', jade_goody),
    ('lucy', lucy_in_the_sky_with_diamonds),
    ('turqoise shelly', turquoise_shelly),
]

In [15]:
solve(grid_1, cats)

(f)oxy lady
(b)lack panther
(j)ade goody
(l)ucy
(t)urqoise shelly


[['t' 't' 'j' 'j' 'l']
 ['t' 'j' 'j' 'j' 'l']
 ['f' 'f' 'b' 'j' 'b']
 ['f' 'f' 'b' 'b' 'b']
 ['f' 'f' 'b' 'b' 'b']]
